In [1]:
# Importing libraries
import os
import pandas as pd
import numpy as np
import sklearn.linear_model
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import metrics
from sklearn import preprocessing
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC, LinearSVC
from sklearn.semi_supervised import label_propagation

# Import miTranscriptome Data (Deprecated)

In [ ]:
# lncrna_exp = pd.read_table("../data/raw/MiTranscriptome-SKCM_lncRNA_exp/mitranscriptome.expr.fpkm.tsv", delimiter="\t")

In [130]:
# lncrna_exp.index = lncrna_exp["transcript_id"]
# lncrna_exp = lncrna_exp.iloc[1:,:]
# lncrna_exp = lncrna_exp.T
lncrna_exp

,Gene_ID,SKCM-Tumor-TCGA-D3-A1Q1,SKCM-Tumor-TCGA-D3-A1Q3,SKCM-Tumor-TCGA-D3-A1Q4,SKCM-Tumor-TCGA-D3-A1Q5,SKCM-Tumor-TCGA-D3-A1Q6,SKCM-Tumor-TCGA-D3-A1Q7,SKCM-Tumor-TCGA-D3-A1Q8,SKCM-Tumor-TCGA-D3-A1Q9,SKCM-Tumor-TCGA-D3-A1QA,...,SKCM-Tumor-TCGA-GF-A3OT,SKCM-Tumor-TCGA-GN-A261,SKCM-Tumor-TCGA-GN-A262,SKCM-Tumor-TCGA-GN-A264,SKCM-Tumor-TCGA-GN-A265,SKCM-Tumor-TCGA-GN-A266,SKCM-Tumor-TCGA-GN-A267,SKCM-Tumor-TCGA-GN-A268,SKCM-Tumor-TCGA-GN-A26A,SKCM-Tumor-TCGA-GN-A26D
0,ENSG00000005206.12,0.221485,0.301805,0.390620,1.380212,0.108369,0.568456,0.317182,0.049637,0.361728,...,0.310220,0.704263,0.918968,0.407143,0.351461,0.881316,1.137700,0.809325,0.480373,0.318671
1,ENSG00000031544.10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035605,0.000000,0.000000,...,0.017139,0.030989,0.000000,0.015993,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,ENSG00000083622.8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.014868,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,ENSG00000088970.11,0.135885,0.053352,0.091178,0.264219,0.126814,0.068246,0.080199,0.038024,0.116305,...,0.000323,0.403619,0.145616,0.174845,0.077805,0.117024,0.045492,0.023763,0.030293,0.105355
4,ENSG00000099869.6,0.000000,0.000000,0.021028,0.000000,0.000000,0.000000,0.021514,0.000000,0.000000,...,0.041424,0.509890,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030828,0.000000
5,ENSG00000100181.17,0.066190,0.063315,2.161100,4.791449,0.470844,0.115449,0.131042,0.014968,0.022503,...,0.093135,0.561728,0.012476,0.073439,0.154906,0.184371,0.031762,0.012495,0.158829,0.018362
6,ENSG00000104691.10,0.157255,0.170210,0.432922,0.526319,0.422747,0.167906,0.218024,0.053316,0.267351,...,0.130371,0.115324,0.379136,0.153083,0.073951,0.124153,0.478343,0.233748,0.664933,0.364459
7,ENSG00000110347.7,0.000000,0.015005,0.020663,0.000000,0.000000,0.113145,0.000000,0.038638,0.000000,...,0.000000,0.000000,0.000000,0.018992,0.000000,0.203660,0.000000,0.000000,0.302928,0.000000
8,ENSG00000115934.11,0.000000,0.000000,0.000000,0.000000,0.000000,0.004087,0.000000,0.003402,0.000000,...,0.000000,0.012619,0.003913,0.000000,0.000000,0.000000,0.000000,0.000000,0.019694,0.000000
9,ENSG00000117242.7,0.024147,0.089057,0.050449,0.356765,0.081990,0.057307,0.048321,0.072257,0.040198,...,0.038062,0.354938,0.205212,0.134669,0.038843,0.255237,0.000000,0.038885,0.018884,0.000000


In [46]:
library_info = pd.read_table("../data/raw/MiTranscriptome-SKCM_lncRNA_exp/library_info.txt", delimiter="\t")
library_info = library_info[library_info["tcga_cancer_type"] == "SKCM"]
library_info["tcga_legacy_sample_id"] = library_info["tcga_legacy_sample_id"].str.slice(0,12)
print list(library_info.columns)
# library_info.head()
print library_info.shape
# library_info[library_info["study_id"] == "uc010roh.3"]
# library_info["sample_type"].unique()
library_info

['library_id', 'study_id', 'cohort_id', 'species', 'library_protocol', 'fragment_layout', 'cancer_progression', 'cancer_type', 'harvest_site', 'sample_type', 'tcga_cancer_type', 'tcga_legacy_sample_id', 'total_fragments', 'frag_size_mean', 'frag_size_stdev', 'total_aligned_reads', 'mean_read_length', 'aligned_bases', 'pct_mrna_bases', 'pct_intronic_bases', 'pct_intergenic_bases']
(265, 21)


,library_id,study_id,cohort_id,species,library_protocol,fragment_layout,cancer_progression,cancer_type,harvest_site,sample_type,...,tcga_legacy_sample_id,total_fragments,frag_size_mean,frag_size_stdev,total_aligned_reads,mean_read_length,aligned_bases,pct_mrna_bases,pct_intronic_bases,pct_intergenic_bases
5148,aaea9670-416a-4293-af12-8bfeb47b0150,phs000178,skin,human,polya_transcriptome,paired,cancer,melanoma,primary_tumor,tissue,...,TCGA-GN-A263,59610748,176,52,90405894,48,4338917038,0.906365,0.040369,0.053261
5149,73b2e2ac-aafd-40e7-8759-ffa226f04a3b,phs000178,skin,human,polya_transcriptome,paired,cancer,melanoma,primary_tumor,tissue,...,TCGA-BF-A1PX,87929125,189,55,148097529,48,7108119385,0.920194,0.020730,0.059075
5150,ca667c68-e9fb-4003-8639-5fa1d00a7a95,phs000178,skin,human,polya_transcriptome,paired,cancer,melanoma,primary_tumor,tissue,...,TCGA-BF-A3DM,74273837,190,57,138166617,48,6631431000,0.929954,0.024018,0.046026
5151,132fbce2-00f3-4ae7-ba18-03bd55529d5a,phs000178,skin,human,polya_transcriptome,paired,cancer,melanoma,primary_tumor,tissue,...,TCGA-EB-A1NK,105410626,188,54,186153542,48,8934650872,0.941784,0.021855,0.036361
5152,5798deb1-2248-474f-b0cd-20c966128d9d,phs000178,skin,human,polya_transcriptome,paired,cancer,melanoma,primary_tumor,tissue,...,TCGA-GF-A2C7,86979753,184,57,151531306,48,7272806761,0.901976,0.048794,0.049228
5153,376614bb-480e-4a7f-9539-da6ce053988a,phs000178,skin,human,polya_transcriptome,paired,cancer,melanoma,primary_tumor,tissue,...,TCGA-ER-A194,69794615,170,48,95875022,48,4601696158,0.937451,0.023451,0.039090
5154,cc0cd3e6-a133-4796-ab06-5adb905d5184,phs000178,skin,human,polya_transcriptome,paired,cancer,melanoma,primary_tumor,tissue,...,TCGA-EB-A24C,90679165,178,55,164459829,48,7893331953,0.909517,0.037165,0.053317
5155,9ff4d588-da73-4e26-975d-2b098fe74984,phs000178,skin,human,polya_transcriptome,paired,cancer,melanoma,primary_tumor,tissue,...,TCGA-BF-A1PU,85539875,181,55,154298231,48,7405681039,0.933139,0.022900,0.043960
5156,27bf4227-d8a9-4048-af7e-01c6d5d0e777,phs000178,skin,human,polya_transcriptome,paired,cancer,melanoma,primary_tumor,tissue,...,TCGA-BF-A1PZ,85121939,173,55,155469327,48,7461917475,0.928395,0.029559,0.042046
5157,8B4C42A9-7D09-4989-A0E2-43ABB6097137,phs000178,skin,human,polya_transcriptome,paired,cancer,melanoma,primary_tumor,tissue,...,TCGA-EB-A3XC,47561588,192,60,89457336,48,4293611333,0.927555,0.028934,0.043510


In [8]:
lncrna_id = pd.read_csv("../data/raw/MiTranscriptome-SKCM_lncRNA_exp/SKCM_lncrna_id.csv")
lncrna_dict = pd.Series(lncrna_id.Name.values, index=lncrna_id["Transcript ID"]).to_dict()
lncrna_exp.replace({"transcript_id": lncrna_dict}, inplace=True)

{'T054686': 'SBF2-AS1.5',
 'T054687': 'SBF2-AS1.6',
 'T232277': 'KCCAT194',
 'T190264': 'KCCAT636',
 'T148754': 'PRCAT315',
 'T314979': 'MEAT52.2',
 'T141422': 'CAT1901.2',
 'T181493': 'AMAT90',
 'T341075': 'CVAT21',
 'T275568': 'AMAT84',
 'T135202': 'LSCAT282',
 'T019989': 'CAT107',
 'T367646': 'THCAT601',
 'T307419': 'CAT833',
 'T188502': 'CAT236',
 'T146596': 'AMAT106',
 'T146597': 'CAT1928',
 'T203739': 'GBAT21',
 'T231786': 'KHCAT171',
 'T107549': 'CAT1691',
 'T252716': 'CAT458',
 'T165229': 'BRCAT57',
 'T299288': 'KCCAT112',
 'T193444': 'LINC00152.1',
 'T028139': 'PCAT6',
 'T074554': 'CAT1479',
 'T148999': 'KCCAT596',
 'T045632': 'NUTM2A-AS1.2',
 'T193443': 'LINC00152.2',
 'T252882': 'LACAT207',
 'T073015': 'CAT1466.2',
 'T020194': 'HRAT4',
 'T205083': 'LACAT88',
 'T327855': 'CAT950',
 'T218701': 'BRCAT322',
 'T150288': 'CAT1950.1',
 'T044850': 'CAT1283',
 'T317967': 'LVCAT6',
 'T073019': 'CAT1466.3',
 'T177396': 'CAT2113.1',
 'T205779': 'CFLAR-AS1.1',
 'T232309': 'CAT2238',
 'T0

In [22]:
lncrna_exp = lncrna_exp.filter(items=["transcript_id",]+list(library_info["library_id"]))

In [31]:
lncrna_exp.index = lncrna_exp["transcript_id"]
lncrna_exp = lncrna_exp.drop("transcript_id", axis=1).T
lncrna_exp['bcr_patient_barcode'] = lncrna_exp.index
lncrna_exp.reset_index(inplace=True)

In [52]:
lncrna_exp.replace({"bcr_patient_barcode": pd.Series(library_info.tcga_legacy_sample_id.values, index=library_info.library_id).to_dict()}, 
                  inplace=True)

In [421]:
lncrna_exp

,Gene_ID,SKCM-Tumor-TCGA-D3-A1Q1,SKCM-Tumor-TCGA-D3-A1Q3,SKCM-Tumor-TCGA-D3-A1Q4,SKCM-Tumor-TCGA-D3-A1Q5,SKCM-Tumor-TCGA-D3-A1Q6,SKCM-Tumor-TCGA-D3-A1Q7,SKCM-Tumor-TCGA-D3-A1Q8,SKCM-Tumor-TCGA-D3-A1Q9,SKCM-Tumor-TCGA-D3-A1QA,...,SKCM-Tumor-TCGA-GF-A3OT,SKCM-Tumor-TCGA-GN-A261,SKCM-Tumor-TCGA-GN-A262,SKCM-Tumor-TCGA-GN-A264,SKCM-Tumor-TCGA-GN-A265,SKCM-Tumor-TCGA-GN-A266,SKCM-Tumor-TCGA-GN-A267,SKCM-Tumor-TCGA-GN-A268,SKCM-Tumor-TCGA-GN-A26A,SKCM-Tumor-TCGA-GN-A26D
Gene_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000005206,ENSG00000005206,0.221485,0.301805,0.390620,1.380212,0.108369,0.568456,0.317182,0.049637,0.361728,...,0.310220,0.704263,0.918968,0.407143,0.351461,0.881316,1.137700,0.809325,0.480373,0.318671
ENSG00000031544,ENSG00000031544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035605,0.000000,0.000000,...,0.017139,0.030989,0.000000,0.015993,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000083622,ENSG00000083622,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.014868,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000088970,ENSG00000088970,0.135885,0.053352,0.091178,0.264219,0.126814,0.068246,0.080199,0.038024,0.116305,...,0.000323,0.403619,0.145616,0.174845,0.077805,0.117024,0.045492,0.023763,0.030293,0.105355
IGF2-AS,IGF2-AS,0.000000,0.000000,0.021028,0.000000,0.000000,0.000000,0.021514,0.000000,0.000000,...,0.041424,0.509890,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030828,0.000000
ENSG00000100181,ENSG00000100181,0.066190,0.063315,2.161100,4.791449,0.470844,0.115449,0.131042,0.014968,0.022503,...,0.093135,0.561728,0.012476,0.073439,0.154906,0.184371,0.031762,0.012495,0.158829,0.018362
ENSG00000104691,ENSG00000104691,0.157255,0.170210,0.432922,0.526319,0.422747,0.167906,0.218024,0.053316,0.267351,...,0.130371,0.115324,0.379136,0.153083,0.073951,0.124153,0.478343,0.233748,0.664933,0.364459
ENSG00000110347,ENSG00000110347,0.000000,0.015005,0.020663,0.000000,0.000000,0.113145,0.000000,0.038638,0.000000,...,0.000000,0.000000,0.000000,0.018992,0.000000,0.203660,0.000000,0.000000,0.302928,0.000000
ENSG00000115934,ENSG00000115934,0.000000,0.000000,0.000000,0.000000,0.000000,0.004087,0.000000,0.003402,0.000000,...,0.000000,0.012619,0.003913,0.000000,0.000000,0.000000,0.000000,0.000000,0.019694,0.000000


# Import lncRNA ENSEMBL nonemclature (TANRIC dataset)

In [3]:
lncrna_exp = pd.read_table("../data/raw/TANRIC-SKCM-lncRNA_exp/TCGA-SKCM-rnaexpr.tsv", delimiter="\t")

In [4]:
lncrna_names = pd.read_table("../data/external/HGNC_RNA_long_non-coding.txt", delimiter="\t")

In [5]:
lncrna_dict = pd.Series(lncrna_names.symbol.values, index=lncrna_names.ensembl_gene_id).to_dict()
lncrna_dict["ENSG00000274721"] = "SPRY4-IT1" # Chr5:142317620-142318322
lncrna_dict["ENSG00000240498"] = "ANRIL" # Chr9:21994791-22121097 (selected 9:21994791-22044790 because 50kb max)
# # lncrna_dict[""] = "RMEL3" Exon1 (Chr5:56690887-56691005)
lncrna_dict["ENSG00000203875"] = "SNHG5" # Chr6:85677007-85678733
# # lncrna_dict[""] = "SLNCR1" # Chr17:72403322-72592804 (selected Chr17:72403322-72453321 because 50kb max)
lncrna_dict["ENSG00000116560"] = "LLME23" # Chr6:165533548-165533786
# "BANCR" # Chr9:69296682-69,306977 

In [6]:
# Replacing ENSG Gene ID to the lncRNA symbol name
lncrna_exp['Gene_ID'] = lncrna_exp['Gene_ID'].str.replace("[.].*", "")
lncrna_exp.replace({"Gene_ID": lncrna_dict}, inplace=True)

# Subsetting lncRNA exp to only key lncRNA's

In [7]:
melanoma_lcrna_exp = lncrna_exp

In [8]:
melanoma_lcrna_exp.index = melanoma_lcrna_exp['Gene_ID']
melanoma_lcrna_exp = melanoma_lcrna_exp.T.iloc[1:,:]

# Filter only lnrRNA expressions in the 13 lncRNA biomarkers


In [9]:
melanoma_lncrnas = ["SPRY4-IT1", "BANCR", "HOTAIR", "MALAT1", "UCA1", "CASC15", "ANRIL", "RMEL3", "SNHG5", "SLNCR1", "SAMMSON", "LLME23", "GAS5"]
melanoma_lcrna_exp = melanoma_lcrna_exp.filter(items=melanoma_lncrnas, axis=1)

In [10]:
# Log 2 transform lncRNA expression data
def lncRNA_exp_transform(x):
    if x <= 0:
        return -16.610
    else:
        return np.log2(x)
melanoma_lcrna_exp = melanoma_lcrna_exp.applymap(lncRNA_exp_transform)

In [11]:
melanoma_lcrna_exp["bcr_patient_barcode"] = melanoma_lcrna_exp.index
print len(melanoma_lcrna_exp["bcr_patient_barcode"].unique())
melanoma_lcrna_exp

226


Gene_ID,HOTAIR,MALAT1,UCA1,CASC15,ANRIL,SNHG5,SAMMSON,GAS5,bcr_patient_barcode
SKCM-Tumor-TCGA-D3-A1Q1,-4.954749,1.056937,-16.610000,1.899652,-5.102630,5.885521,-1.183643,6.355847,SKCM-Tumor-TCGA-D3-A1Q1
SKCM-Tumor-TCGA-D3-A1Q3,-1.771717,3.253181,-16.610000,1.308918,-1.862001,4.875168,-2.608419,3.818640,SKCM-Tumor-TCGA-D3-A1Q3
SKCM-Tumor-TCGA-D3-A1Q4,-0.459055,2.744404,-16.610000,0.026315,-2.232588,5.529213,0.345814,5.846049,SKCM-Tumor-TCGA-D3-A1Q4
SKCM-Tumor-TCGA-D3-A1Q5,2.567640,3.084154,-6.813222,0.632330,-16.610000,3.974051,-1.337087,4.911555,SKCM-Tumor-TCGA-D3-A1Q5
SKCM-Tumor-TCGA-D3-A1Q6,0.557473,3.034719,-0.418887,-2.616813,-2.278941,3.084658,-0.754754,4.031756,SKCM-Tumor-TCGA-D3-A1Q6
SKCM-Tumor-TCGA-D3-A1Q7,1.042103,2.726287,-16.610000,-0.417327,-1.004827,4.577221,-0.639667,4.128122,SKCM-Tumor-TCGA-D3-A1Q7
SKCM-Tumor-TCGA-D3-A1Q8,0.379875,1.261015,-16.610000,0.314668,-7.525788,3.679479,-1.707020,3.419743,SKCM-Tumor-TCGA-D3-A1Q8
SKCM-Tumor-TCGA-D3-A1Q9,-5.869329,3.063182,-6.105752,0.152281,-5.052566,3.727757,2.752284,3.260470,SKCM-Tumor-TCGA-D3-A1Q9
SKCM-Tumor-TCGA-D3-A1QA,-4.156082,1.524015,-7.834412,0.308342,-1.510868,5.054624,0.447697,5.949019,SKCM-Tumor-TCGA-D3-A1QA
SKCM-Tumor-TCGA-D3-A1QB,-5.855862,0.600855,-7.646874,0.488985,-4.473526,3.830398,-1.089961,3.943666,SKCM-Tumor-TCGA-D3-A1QB


# Adding exp of missing lncRNAs

In [12]:
# lncrna_exp = pd.read_table("../data/raw/TANRIC-SKCM-lncRNA_exp/lncRNA_exp_by_genomic_coordinate/ANRIL_log2exp_SKCM.tsv", delimiter="\t")
for fn in os.listdir('../data/raw/TANRIC-SKCM-lncRNA_exp/lncRNA_exp_by_genomic_coordinate/'):
    if len(str.split(fn, ".")) > 1 and str.split(fn, ".")[1] == "tsv":
        lncRNA_name = str.split(fn, "_")[0]
        missing_lncRNA_exp = pd.read_table("../data/raw/TANRIC-SKCM-lncRNA_exp/lncRNA_exp_by_genomic_coordinate/"+fn, delimiter="\t")
        print lncRNA_name, missing_lncRNA_exp.shape
        missing_lncRNA_exp.rename(columns={"lncRNA_exp_log2": lncRNA_name}, inplace=True)
        melanoma_lcrna_exp = melanoma_lcrna_exp.merge(missing_lncRNA_exp.drop(["Cancer", "position_annotation"], axis=1), 
                                       how="outer", left_on="bcr_patient_barcode", right_on="SampleID", suffixes=('','_y'))

ANRIL (226, 4)
SPRY4-IT1 (226, 4)
SLNCR1 (226, 4)
SNHG5 (226, 4)
LLME23 (226, 4)


IndexError: list index out of range

In [13]:
# melanoma_lcrna_exp.drop([x for x in melanoma_lcrna_exp.columns if x.endswith('SampleID_y')], axis=1, inplace=True)
# melanoma_lcrna_exp.drop([x for x in melanoma_lcrna_exp.columns if x.endswith('SNHG5_y')], axis=1, inplace=True)
melanoma_lcrna_exp.drop([x for x in melanoma_lcrna_exp.columns if x.endswith('_y')], axis=1, inplace=True)

In [14]:
print melanoma_lcrna_exp.columns
melanoma_lcrna_exp

Index([u'HOTAIR', u'MALAT1', u'UCA1', u'CASC15', u'ANRIL', u'SNHG5',
       u'SAMMSON', u'GAS5', u'bcr_patient_barcode', u'SampleID', u'SPRY4-IT1',
       u'SLNCR1', u'LLME23'],
      dtype='object')


,HOTAIR,MALAT1,UCA1,CASC15,ANRIL,SNHG5,SAMMSON,GAS5,bcr_patient_barcode,SampleID,SPRY4-IT1,SLNCR1,LLME23
0,-4.954749,1.056937,-16.610000,1.899652,-5.102630,5.885521,-1.183643,6.355847,SKCM-Tumor-TCGA-D3-A1Q1,SKCM-Tumor-TCGA-D3-A1Q1,-16.610,-5.374,-16.610
1,-1.771717,3.253181,-16.610000,1.308918,-1.862001,4.875168,-2.608419,3.818640,SKCM-Tumor-TCGA-D3-A1Q3,SKCM-Tumor-TCGA-D3-A1Q3,-16.610,-5.454,-3.866
2,-0.459055,2.744404,-16.610000,0.026315,-2.232588,5.529213,0.345814,5.846049,SKCM-Tumor-TCGA-D3-A1Q4,SKCM-Tumor-TCGA-D3-A1Q4,-16.610,-2.095,-16.610
3,2.567640,3.084154,-6.813222,0.632330,-16.610000,3.974051,-1.337087,4.911555,SKCM-Tumor-TCGA-D3-A1Q5,SKCM-Tumor-TCGA-D3-A1Q5,-16.610,-1.245,-16.610
4,0.557473,3.034719,-0.418887,-2.616813,-2.278941,3.084658,-0.754754,4.031756,SKCM-Tumor-TCGA-D3-A1Q6,SKCM-Tumor-TCGA-D3-A1Q6,-16.610,-6.271,-6.307
5,1.042103,2.726287,-16.610000,-0.417327,-1.004827,4.577221,-0.639667,4.128122,SKCM-Tumor-TCGA-D3-A1Q7,SKCM-Tumor-TCGA-D3-A1Q7,-16.610,-2.145,-16.610
6,0.379875,1.261015,-16.610000,0.314668,-7.525788,3.679479,-1.707020,3.419743,SKCM-Tumor-TCGA-D3-A1Q8,SKCM-Tumor-TCGA-D3-A1Q8,-16.610,-1.140,-16.610
7,-5.869329,3.063182,-6.105752,0.152281,-5.052566,3.727757,2.752284,3.260470,SKCM-Tumor-TCGA-D3-A1Q9,SKCM-Tumor-TCGA-D3-A1Q9,-16.610,-4.571,-16.610
8,-4.156082,1.524015,-7.834412,0.308342,-1.510868,5.054624,0.447697,5.949019,SKCM-Tumor-TCGA-D3-A1QA,SKCM-Tumor-TCGA-D3-A1QA,-16.610,-5.669,-8.563
9,-5.855862,0.600855,-7.646874,0.488985,-4.473526,3.830398,-1.089961,3.943666,SKCM-Tumor-TCGA-D3-A1QB,SKCM-Tumor-TCGA-D3-A1QB,-16.610,-5.909,-16.610


In [15]:
melanoma_lcrna_exp["bcr_patient_barcode"] = melanoma_lcrna_exp["bcr_patient_barcode"].str.replace("SKCM-Tumor-", "")

In [16]:
# Missing lncRNAs
set(melanoma_lncrnas) - set(melanoma_lcrna_exp.columns.tolist())

{'BANCR', 'RMEL3'}

# Importing all TCGA SKCM patient's clinical data
## Doesn't necessary include all SKCM patients with lncRNA expressions profiled

In [125]:
skcm_clinical = pd.read_table("../data/processed/SKCM_clinical/clinical_SKCM_all.txt__nationwidechildrens.org_clinical_drug_skcm.txt", 
                             delimiter="\t")
skcm_clinical = skcm_clinical[['bcr_patient_barcode', 'pharmaceutical_therapy_type', 'treatment_best_response']].iloc[1:,:]
np.unique(skcm_clinical['treatment_best_response'])

array(['CDE_ID:2857291', 'Clinical Progressive Disease',
       'Complete Response', 'Partial Response', 'Stable Disease',
       '[Not Applicable]', '[Not Available]', '[Unknown]'], dtype=object)

In [126]:
skcm_clinical = skcm_clinical[skcm_clinical["pharmaceutical_therapy_type"] == "Immunotherapy"]
skcm_clinical.replace({"treatment_best_response":{"[Unknown]":-1, "[Not Available]":-1, "[Not Applicable]":-1}}, 
                      inplace=True)

skcm_clinical = skcm_clinical[skcm_clinical["treatment_best_response"].isin(['Clinical Progressive Disease', 
                                                                             'Complete Response', 
                                                                             'Partial Response', 
                                                                             'Stable Disease',
                                                                             'CDE_ID:2857291'])]

print skcm_clinical["treatment_best_response"].value_counts()

Clinical Progressive Disease    20
Complete Response               12
Partial Response                 9
Stable Disease                   8
Name: treatment_best_response, dtype: int64


In [127]:
print skcm_clinical.shape
skcm_clinical

(49, 3)


,bcr_patient_barcode,pharmaceutical_therapy_type,treatment_best_response
3,TCGA-D3-A1Q1,Immunotherapy,Clinical Progressive Disease
15,TCGA-D3-A2JP,Immunotherapy,Partial Response
19,TCGA-D3-A3MV,Immunotherapy,Clinical Progressive Disease
30,TCGA-D3-A8GB,Immunotherapy,Complete Response
32,TCGA-D3-A8GN,Immunotherapy,Complete Response
35,TCGA-DA-A3F2,Immunotherapy,Partial Response
37,TCGA-DA-A3F5,Immunotherapy,Stable Disease
62,TCGA-EB-A57M,Immunotherapy,Complete Response
69,TCGA-EB-A5KH,Immunotherapy,Complete Response
72,TCGA-EB-A5SH,Immunotherapy,Complete Response


# Merging lncRNA expression data to clinical data, select only samples with immunotherapy responses

In [128]:
melanoma_lcrna_immuno_exp = melanoma_lcrna_exp.merge(skcm_clinical, how="inner", 
                                                     left_on="bcr_patient_barcode", right_on="bcr_patient_barcode")

In [129]:
# melanoma_lcrna_immuno_exp[melanoma_lcrna_immuno_exp["treatment_best_response"].notnull()]
melanoma_lcrna_immuno_exp.dropna(axis=1, inplace=True)
melanoma_lcrna_immuno_exp

,HOTAIR,MALAT1,UCA1,CASC15,ANRIL,SNHG5,SAMMSON,GAS5,bcr_patient_barcode,SampleID,SPRY4-IT1,SLNCR1,LLME23,pharmaceutical_therapy_type,treatment_best_response
0,-4.954749,1.056937,-16.610000,1.899652,-5.102630,5.885521,-1.183643,6.355847,TCGA-D3-A1Q1,SKCM-Tumor-TCGA-D3-A1Q1,-16.610,-5.374,-16.610,Immunotherapy,Clinical Progressive Disease
1,-4.187627,3.932285,-16.610000,-1.025823,-1.792554,5.649553,-1.260712,5.198739,TCGA-D3-A2JP,SKCM-Tumor-TCGA-D3-A2JP,-16.610,-6.427,-5.443,Immunotherapy,Partial Response
2,-0.180198,2.915019,-6.748868,1.266341,-1.905685,3.344050,1.794626,5.455271,TCGA-D3-A3MV,SKCM-Tumor-TCGA-D3-A3MV,-16.610,-6.479,-5.482,Immunotherapy,Clinical Progressive Disease
3,-5.395841,0.253219,-16.610000,0.035796,-5.162393,2.874259,-1.910087,3.422120,TCGA-EE-A29C,SKCM-Tumor-TCGA-EE-A29C,-16.610,-3.675,-16.610,Immunotherapy,Stable Disease
4,0.774323,2.107389,-16.610000,0.963327,-8.231940,5.258453,0.245759,5.576159,TCGA-EE-A2GS,SKCM-Tumor-TCGA-EE-A2GS,-16.610,-1.509,-5.643,Immunotherapy,Clinical Progressive Disease
5,-2.756283,2.768896,-7.387705,-0.322126,-7.352771,4.460757,-0.978525,4.949581,TCGA-EE-A2ML,SKCM-Tumor-TCGA-EE-A2ML,-6.677,-6.659,-3.006,Immunotherapy,Clinical Progressive Disease
6,-1.394977,4.361727,-16.610000,-4.102879,-0.450310,5.493224,0.557163,6.731395,TCGA-EE-A3JI,SKCM-Tumor-TCGA-EE-A3JI,-6.141,-8.203,-16.610,Immunotherapy,Clinical Progressive Disease
7,-2.854754,2.512424,-16.610000,0.766727,-6.179518,3.043991,1.434712,3.857484,TCGA-ER-A19M,SKCM-Tumor-TCGA-ER-A19M,-16.610,-5.553,-3.590,Immunotherapy,Clinical Progressive Disease
8,-3.055863,2.333020,-16.610000,-0.822680,-9.377590,4.878814,0.548832,5.087172,TCGA-FR-A3YN,SKCM-Tumor-TCGA-FR-A3YN,-16.610,-8.226,-1.718,Immunotherapy,Complete Response
9,-4.987265,-1.609988,-5.378506,-2.774487,-6.799251,4.721482,-5.654659,3.983923,TCGA-FR-A3YO,SKCM-Tumor-TCGA-FR-A3YO,-16.610,-1.130,-16.610,Immunotherapy,Stable Disease


# Importing TCGA Gene Expression data (Filter Immune Genes from TCIA)

In [130]:
skcm_gene_exp = pd.read_table("../data/processed/SKCM__geneExp_normalized.txt", delimiter="\t")
skcm_gene_exp.drop(["EntrezID"], axis=1, inplace=True)
skcm_gene_exp = skcm_gene_exp[skcm_gene_exp["GeneSymbol"] != "?"]
skcm_gene_exp.index = skcm_gene_exp['GeneSymbol']
skcm_gene_exp = skcm_gene_exp.T.iloc[1:,:]

In [131]:
_, i = np.unique(skcm_gene_exp.columns, return_index=True)
skcm_gene_exp = skcm_gene_exp.iloc[:, i]

In [132]:
immune_genes = pd.read_table("../data/processed/TCIA-GeneExpressionHeatmap_SKCM.csv", delimiter="\t")["Genes"]

In [133]:
skcm_gene_exp = skcm_gene_exp.filter(immune_genes.tolist(), axis=1)

In [134]:
skcm_gene_exp["bcr_patient_barcode"] = skcm_gene_exp.index

In [135]:
skcm_gene_exp["bcr_patient_barcode"] = skcm_gene_exp["bcr_patient_barcode"].str[0:12]

# Label Propagation for Semi-Supervised Learning from Immune Gene Expressions

In [136]:
melanoma_lcrna_immuno_exp["treatment_best_response"].value_counts()

Clinical Progressive Disease    8
Stable Disease                  5
Complete Response               2
Partial Response                1
Name: treatment_best_response, dtype: int64

In [137]:
X = melanoma_lcrna_immuno_exp[["bcr_patient_barcode", "treatment_best_response"]].merge(skcm_gene_exp, 
                                                                                    left_on="bcr_patient_barcode",
                                                                                    right_on="bcr_patient_barcode",
                                                                                       ).drop(["bcr_patient_barcode", "treatment_best_response"], axis=1)
X

,ZNF645,ZNF165,ZEB2,ZCRB1,ZCCHC2,ZAP70,YBX2,XRCC6,XPO6,XAGE5,...,ACRBP,ACP5,ACAP1,ACADM,ABTB1,ABHD3,ABCD1,ABCB1,ABCA1,ABAT
0,0,22.9585,3270.77,311.605,230.301,3.2235,3.9398,10945.6,2891.83,0,...,33.3095,2450.93,6.447,916.905,127.507,167.98,239.613,80.5874,429.43,45.4871
1,0,16.5441,1806.62,746.691,124.632,162.5,1.1029,7943.38,1750.74,0,...,15.0735,2461.4,180.147,653.309,615.441,437.132,3268.38,24.6324,911.393,45.5882
2,0,25.36,2399.42,894.26,209.27,12.0512,11.7422,12542.8,2869.42,31.2095,...,35.2266,3712.08,31.8275,641.185,576.603,85.9034,469.996,61.183,1102.83,45.7327
3,0,14.1356,1984.85,320.209,249.215,28.2049,4.4243,6494.32,4227.97,0,...,10.5077,9259.51,65.2584,502.711,504.37,246.655,1723.82,17.1442,919.149,47.5612
4,0,14.8368,2425.22,694.362,397.323,39.7626,20.178,10052.5,3136.2,1.1869,...,24.3323,4710.98,56.9733,909.495,380.712,144.51,721.068,10.6825,375.071,18.9911
5,0,10.5943,4894.57,738.712,841.964,208.355,0.6421,5228.45,2344.87,0.321,...,15.0889,2736.54,228.259,973.07,854.928,342.549,563.103,63.8868,351.538,623.458
6,0.2494,23.0923,3987.53,315.461,356.529,33.4165,0.2494,5240.15,1442.14,0.2494,...,20.4489,215.212,38.6534,1236.91,754.863,457.855,829.426,7.2319,116.187,47.1322
7,0,11.0377,2364.32,1017.54,248.227,192.982,4.8526,10281.4,3679.36,0,...,36.5808,6565.14,203.061,972.378,216.872,355.73,1002.24,175.812,852.93,213.886
8,0,8.938,4393.37,821.356,317.029,125.982,0,7508,2223.16,0,...,16.5842,3283.39,164.679,2074.77,650.858,192.901,2851.91,142.857,1501.89,34.3323
9,0,2.5063,948.371,199.499,104.632,4.5113,27.0677,9426.07,3265.16,4.01,...,3.5088,11230.6,44.6115,251.629,1182.46,83.7093,22171.4,189.975,290.216,43.609


In [138]:
y_semi = melanoma_lcrna_immuno_exp["treatment_best_response"]
label_prop_model = label_propagation.LabelPropagation(kernel='rbf', n_neighbors=1, gamma=1, alpha=0.5)
label_prop_model.fit(X, y_semi)
y_semi_predicted = pd.Series(label_prop_model.predict(X))
y_semi_predicted.value_counts()

Clinical Progressive Disease    8
Stable Disease                  5
Complete Response               2
Partial Response                1
dtype: int64

# Prepare X, y for classification 

In [21]:
X = melanoma_lcrna_immuno_exp.drop(['bcr_patient_barcode', 'SampleID', 'pharmaceutical_therapy_type', 'treatment_best_response'], axis=1
X
# X = X.applymap(lncRNA_exp_transform, ) # Log2 transform lncRNA expressions

,HOTAIR,MALAT1,UCA1,CASC15,ANRIL,SNHG5,SAMMSON,GAS5,LLME23,RMEL3,SLNCR1,SPRY4-IT1
0,-4.954749,1.056937,-16.610000,1.899652,-5.102630,5.885521,-1.183643,6.355847,-16.610,-16.610,-5.374,-16.610
1,-4.187627,3.932285,-16.610000,-1.025823,-1.792554,5.649553,-1.260712,5.198739,-5.443,-16.610,-6.427,-16.610
2,-0.180198,2.915019,-6.748868,1.266341,-1.905685,3.344050,1.794626,5.455271,-5.482,-16.610,-6.479,-16.610
3,-5.395841,0.253219,-16.610000,0.035796,-5.162393,2.874259,-1.910087,3.422120,-16.610,-16.610,-3.675,-16.610
4,0.774323,2.107389,-16.610000,0.963327,-8.231940,5.258453,0.245759,5.576159,-5.643,-3.410,-1.509,-16.610
5,-2.756283,2.768896,-7.387705,-0.322126,-7.352771,4.460757,-0.978525,4.949581,-3.006,-4.115,-6.659,-6.677
6,-1.394977,4.361727,-16.610000,-4.102879,-0.450310,5.493224,0.557163,6.731395,-16.610,-8.160,-8.203,-6.141
7,-2.854754,2.512424,-16.610000,0.766727,-6.179518,3.043991,1.434712,3.857484,-3.590,1.752,-5.553,-16.610
8,-3.055863,2.333020,-16.610000,-0.822680,-9.377590,4.878814,0.548832,5.087172,-1.718,-16.610,-8.226,-16.610
9,-4.987265,-1.609988,-5.378506,-2.774487,-6.799251,4.721482,-5.654659,3.983923,-16.610,-16.610,-1.130,-16.610


In [139]:
# y = melanoma_lcrna_immuno_exp["treatment_best_response"].as_matrix()
# y = y_semi.as_matrix()
y = melanoma_lcrna_immuno_exp.replace({"treatment_best_response":{"Clinical Progressive Disease":-1, 
                                                               "Stable Disease":0, 
                                                               "Partial Response":1, 
                                                               "Complete Response":2}}, 
                                      inplace=False)["treatment_best_response"].as_matrix()
y

array([-1,  1, -1,  0, -1, -1, -1, -1,  2,  0,  0,  0,  0,  2, -1, -1])

In [140]:
lncrna_list = X.columns
lncrna_list

Index([u'ZNF645', u'ZNF165', u'ZEB2', u'ZCRB1', u'ZCCHC2', u'ZAP70', u'YBX2',
       u'XRCC6', u'XPO6', u'XAGE5',
       ...
       u'ACRBP', u'ACP5', u'ACAP1', u'ACADM', u'ABTB1', u'ABHD3', u'ABCD1',
       u'ABCB1', u'ABCA1', u'ABAT'],
      dtype='object', length=1012)

# Using expression array to predict treatment response

In [141]:
# model = sklearn.linear_model.Lasso(alpha=1e-1, fit_intercept=False, copy_X=True, normalize=True, max_iter=2000)
# model = sklearn.linear_model.LogisticRegression(C=5e-2, penalty='l1', fit_intercept=False, class_weight="balanced")
model = LinearSVC(C=1e-5, penalty='l1', class_weight='balanced', dual=False, multi_class="ovr")
# model = SVC(C=1e0, kernel="rbf", class_weight="balanced", decision_function_shape="ovo")
# model = DecisionTreeClassifier(criterion='gini', splitter='best', max_features=2, max_depth=2, class_weight="balanced")
# model = sklearn.linear_model.LassoCV(cv=2, fit_intercept=False, copy_X=True, normalize=True, max_iter=2000)

# model.fit(X=preprocessing.scale(X=X, with_mean=True, with_std=False),y=y)
model.fit(X=X, y=y)

print "coef:", len(np.nonzero(model.coef_)[0])

print metrics.classification_report(y_pred=np.asanyarray(np.round(model.predict(X)), dtype=int), y_true=y)
print metrics.confusion_matrix(y_pred=np.asanyarray(np.round(model.predict(X)), dtype=int), y_true=y)
# print metrics.classification_report(y_pred=model.predict(X), y_true=y)
# print metrics.confusion_matrix(y_pred=model.predict(X), y_true=y)

coef: 23
             precision    recall  f1-score   support

         -1       0.89      1.00      0.94         8
          0       1.00      0.80      0.89         5
          1       1.00      1.00      1.00         1
          2       1.00      1.00      1.00         2

avg / total       0.94      0.94      0.94        16

[[8 0 0 0]
 [1 4 0 0]
 [0 0 1 0]
 [0 0 0 2]]


# View selected lncRNA's

In [142]:
coefs = np.sum(np.abs(model.coef_), axis=0)

# coefs = model.coef_
lncrna_list[np.nonzero(coefs)]

Index([u'VIM', u'SPP1', u'PSAP', u'MBP', u'IGFBP5', u'HLA-B', u'HLA-A', u'FN1',
       u'CTSD', u'CD63', u'B2M', u'ANXA5', u'ACTG1'],
      dtype='object')

In [143]:
# candidate_lncrna_list = lncrnas_available[sfm.get_support()].tolist()
candidate_lncrna_list = lncrna_list[np.nonzero(coefs)]

candidate_lncrna = pd.DataFrame()
candidate_lncrna["lncrna"] = candidate_lncrna_list
candidate_lncrna["coef"] = coefs[np.nonzero(coefs)]
# candidate_lncrna.reindex(candidate_lncrna.coef.abs().sort_values(inplace=False).index)
candidate_lncrna = candidate_lncrna.reindex(candidate_lncrna["coef"].abs().sort_values(ascending=False,inplace=False).index)
candidate_lncrna
# candidate_lncrna.sort_values(by="coef", ascending=False)

,lncrna,coef
1,SPP1,0.000024
12,ACTG1,0.000017
6,HLA-A,0.000011
3,MBP,0.000010
7,FN1,0.000008
9,CD63,0.000007
0,VIM,0.000007
10,B2M,0.000004
2,PSAP,0.000004
4,IGFBP5,0.000003


# LEAVE ONE OUT CROSSVALIDATION

In [146]:
loo = LeaveOneOut()
correct_count = 0

#     model = sklearn.linear_model.Lasso(alpha=1e0, fit_intercept=False, copy_X=True, normalize=True, max_iter=2000, selection="random")
model = LinearSVC(C=1e-0, penalty='l1', class_weight='balanced', dual=False)
# model = SVC(C=1e0, kernel="rbf", class_weight="balanced", decision_function_shape="ovo", degree=3)
# model = DecisionTreeClassifier(criterion='gini', splitter='best')

for train, test in loo.split(X=X, y=y):
    model.fit(X=X.as_matrix()[train], y=y[train])
#     correct_count += metrics.accuracy_score(y_pred=np.asanyarray(np.round(model.predict(X.as_matrix()[test])), dtype=int), 
#                                             y_true=y[test])
    correct_count += metrics.accuracy_score(y_pred=model.predict(X.as_matrix()[test]), 
                                            y_true=y[test])
    
print "accuracy", correct_count/float(X.shape[0])
print "non_zero", len(np.nonzero(model.coef_)[0])
# print lncrna_list[np.nonzero(model.coef_)[0]]
# model.coef_

accuracy 0.1875
non_zero 755


# TRAIN TEST SPLIT CROSS VALIDATION

In [147]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3)

# model = SVC(C=1e2, kernel="rbf", class_weight="balanced", decision_function_shape="ovr")
model.fit(X=X_train, y=y_train)
# print metrics.classification_report(y_pred=np.asanyarray(np.round(model.predict(X_test)), dtype=int), y_true=y_test)
print metrics.classification_report(y_pred=model.predict(X_test), y_true=y_test)

             precision    recall  f1-score   support

         -1       0.50      0.33      0.40         3
          0       0.00      0.00      0.00         1
          1       0.00      0.00      0.00         0
          2       0.00      0.00      0.00         1

avg / total       0.30      0.20      0.24         5



/opt/miniconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/miniconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# Correlation Analysis of each lncRNA to the labels

To determine the response vector, follow this tumor response criteria

Evaluation of target lesions

Complete Response (CR): Disappearance of all target lesions

Partial Response (PR): At least a 30% decrease in the sum of the LD of target lesions, taking as reference the baseline sum LD

Stable Disease (SD): Neither sufficient shrinkage to qualify for PR nor sufficient increase to qualify for PD, taking as reference the smallest sum LD since the treatment started

Progressive Disease (PD): At least a 20% increase in the sum of the LD of target lesions, taking as reference the smallest sum LD recorded since the treatment started or the appearance of one or more new lesions

In [148]:
lncrnas_available = melanoma_lcrna_immuno_exp.drop(['bcr_patient_barcode', 'pharmaceutical_therapy_type', 'SampleID', 'treatment_best_response'], axis=1).columns

In [149]:
lncrna_list

Index([u'ZNF645', u'ZNF165', u'ZEB2', u'ZCRB1', u'ZCCHC2', u'ZAP70', u'YBX2',
       u'XRCC6', u'XPO6', u'XAGE5',
       ...
       u'ACRBP', u'ACP5', u'ACAP1', u'ACADM', u'ABTB1', u'ABHD3', u'ABCD1',
       u'ABCB1', u'ABCA1', u'ABAT'],
      dtype='object', length=1012)

In [150]:
responses = melanoma_lcrna_immuno_exp.replace({"treatment_best_response":{"Clinical Progressive Disease":-3, 
                                                                          "Stable Disease":0,
                                                                          "Partial Response":3, 
                                                                          "Complete Response":10}}, 
                                              inplace=False)["treatment_best_response"].as_matrix()

# responses = pd.DataFrame(y_semi, columns=["treatment_best_response",]).replace({"treatment_best_response":{"Clinical Progressive Disease":0, 
#                                                                           "Stable Disease":0,
#                                                                           "Partial Response":5, 
#                                                                           "Complete Response":10}}, 
#                                               inplace=False)["treatment_best_response"].as_matrix()

In [153]:
print "(correlation, p-value)\t\t\t\tlncRNA"
for lncrna in lncrna_list:
    if np.abs(stats.pearsonr(X[lncrna].as_matrix(), responses)[0]) > 0.65:
        print stats.pearsonr(X[lncrna].as_matrix(), responses), "\t"+lncrna

(correlation, p-value)				lncRNA
(0.6556078710880846, 0.0058283434261976325) 	WIPF1
(0.69570816301623217, 0.002764533677333867) 	THBD
(0.80982655119710623, 0.00014351608334997975) 	MBP
(0.6973071499381932, 0.002677170031833031) 	ELMOD3
(0.69236901590837896, 0.0029544093777311643) 	EGR3
(0.66726044660084283, 0.0047449403737648953) 	AHCYL1


/opt/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in absolute
  app.launch_new_instance()


# Try sklearn.feature_selection

In [154]:
sfm = SelectFromModel(DecisionTreeClassifier(max_features=5, max_depth=5), threshold=None)
# sfm = d(model, threshold=None, prefit=True)

In [157]:
sfm.fit(X, y)
# sfm.transform(X)
sfm.get_params()
sfm.estimator.get_params()

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': 5,
 'max_leaf_nodes': None,
 'min_impurity_split': 1e-07,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}

In [158]:
sfm.get_support()

array([False, False, False, ..., False, False, False], dtype=bool)

In [75]:
model = sfm.estimator_

AttributeError: 'SelectFromModel' object has no attribute 'estimator_'

In [159]:
lncrnas_available[sfm.get_support()].tolist()

/opt/miniconda2/lib/python2.7/site-packages/pandas/core/indexes/base.py:1650: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 11 but corresponding boolean dimension is 1012
  result = getitem(key)


IndexError: index 66 is out of bounds for axis 1 with size 11

# Plot Data & Decision Bounds

In [166]:
# candidate_lncrna_list = candidate_lncrna["lncrna"].tolist()
# candidate_lncrna_list = lncrnas_available[sfm.get_support()].tolist() # From feature selector
# candidate_lncrna_list = ["LLME23", "SNHG5", "RMEL3", "SPRY4-IT1", "SLNCR1"]
candidate_lncrna_list = ["MBP", "THBD", "ELMOD3", "EGR3", "AHCYL1"]
# candidate_lncrna_list = list(lncrna_list)

features = []
top_candidate_lncrna = candidate_lncrna_list
for i, lncrna1 in enumerate(top_candidate_lncrna):
    for j, lncrna2 in enumerate(top_candidate_lncrna):
        if i < j:
            features.append([lncrna1, lncrna2])
print len(features)
features

10


[['MBP', 'THBD'],
 ['MBP', 'ELMOD3'],
 ['MBP', 'EGR3'],
 ['MBP', 'AHCYL1'],
 ['THBD', 'ELMOD3'],
 ['THBD', 'EGR3'],
 ['THBD', 'AHCYL1'],
 ['ELMOD3', 'EGR3'],
 ['ELMOD3', 'AHCYL1'],
 ['EGR3', 'AHCYL1']]

In [170]:
X

,ZNF645,ZNF165,ZEB2,ZCRB1,ZCCHC2,ZAP70,YBX2,XRCC6,XPO6,XAGE5,...,ACRBP,ACP5,ACAP1,ACADM,ABTB1,ABHD3,ABCD1,ABCB1,ABCA1,ABAT
0,0,22.9585,3270.77,311.605,230.301,3.2235,3.9398,10945.6,2891.83,0,...,33.3095,2450.93,6.447,916.905,127.507,167.98,239.613,80.5874,429.43,45.4871
1,0,16.5441,1806.62,746.691,124.632,162.5,1.1029,7943.38,1750.74,0,...,15.0735,2461.4,180.147,653.309,615.441,437.132,3268.38,24.6324,911.393,45.5882
2,0,25.36,2399.42,894.26,209.27,12.0512,11.7422,12542.8,2869.42,31.2095,...,35.2266,3712.08,31.8275,641.185,576.603,85.9034,469.996,61.183,1102.83,45.7327
3,0,14.1356,1984.85,320.209,249.215,28.2049,4.4243,6494.32,4227.97,0,...,10.5077,9259.51,65.2584,502.711,504.37,246.655,1723.82,17.1442,919.149,47.5612
4,0,14.8368,2425.22,694.362,397.323,39.7626,20.178,10052.5,3136.2,1.1869,...,24.3323,4710.98,56.9733,909.495,380.712,144.51,721.068,10.6825,375.071,18.9911
5,0,10.5943,4894.57,738.712,841.964,208.355,0.6421,5228.45,2344.87,0.321,...,15.0889,2736.54,228.259,973.07,854.928,342.549,563.103,63.8868,351.538,623.458
6,0.2494,23.0923,3987.53,315.461,356.529,33.4165,0.2494,5240.15,1442.14,0.2494,...,20.4489,215.212,38.6534,1236.91,754.863,457.855,829.426,7.2319,116.187,47.1322
7,0,11.0377,2364.32,1017.54,248.227,192.982,4.8526,10281.4,3679.36,0,...,36.5808,6565.14,203.061,972.378,216.872,355.73,1002.24,175.812,852.93,213.886
8,0,8.938,4393.37,821.356,317.029,125.982,0,7508,2223.16,0,...,16.5842,3283.39,164.679,2074.77,650.858,192.901,2851.91,142.857,1501.89,34.3323
9,0,2.5063,948.371,199.499,104.632,4.5113,27.0677,9426.07,3265.16,4.01,...,3.5088,11230.6,44.6115,251.629,1182.46,83.7093,22171.4,189.975,290.216,43.609


In [173]:
for i, axis in enumerate(features):
    X_sub = X[axis]

    # create a mesh to plot in
    h = .2  # step size in the mesh
    x_min, x_max = X_sub[axis[0]].min() - 1, X_sub[axis[0]].max() + 1
    y_min, y_max = X_sub[axis[1]].min() - 1, X_sub[axis[1]].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    plt.figure(figsize=(5,5)) 
#     plt.subplot(6, 6, i+1)
#     plt.subplots_adjust(wspace=0.1, hspace=0.1)

    model.fit(X_sub, y)
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
    accuracy = metrics.accuracy_score(y_pred=np.asanyarray(np.round(model.predict(X_sub)), dtype=int), y_true=y)

    # Plot also the training points
    plt.scatter(X_sub[axis[0]], X_sub[axis[1]], c=y, cmap=plt.cm.coolwarm, edgecolors="black")
    plt.xlabel(axis[0])
    plt.ylabel(axis[1])
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
#     plt.xticks(())
#     plt.yticks(())
#     plt.legend(loc='lower left', numpoints=1, ncol=3, fontsize=8, bbox_to_anchor=(0, 0))
    plt.title("accuracy: "+str(accuracy))
plt.show()

MemoryError: 